In [1]:
# import packages
import numpy as np
import pylab as py
import pdb
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# import nbody6tools reader to read data
from nbody6tools import Reader

# import nbody62spisea to retrieve star ages
from nbody62spisea import converter

/home/wyz5rge/.local/lib/python3.9/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


## Set up parameters, create isochrone grid

In [2]:
# define isochrone parameters
AKs = 0
dist = 410
evo_model = evolution.MISTv1()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F115W', 'jwst,F182M']
filters = ['m_jwst_F115W', 'm_jwst_F182M']
iso_dir = 'isochrones/'
metallicity = 0

In [3]:
# read nbody data
path_to_data = '../../../../../../storage/jfarias/Data/Orion/massive_clusters/M3000new/sigma0p1/fiducial/sfe_ff003/00/'
snapshot = Reader.read_snapshot(path_to_data, time = 20)

# convert snapshot values from nbody units to astrophysical, extract masses
snapshot.to_physical()

# read ages for simulated stars
cluster_table = converter.to_spicea_table(snapshot)

print(cluster_table)

        mass         isMultiple       compMass            systemMass              e                log_a               age        
-------------------- ---------- -------------------- ------------------- ------------------- ------------------ ------------------
  12.818908070209863        0.0                   []  12.818908070209863                 nan                nan  19.28537582919208
  4.9549285763343764        0.0                   []  4.9549285763343764                 nan                nan  19.28537582919208
   8.067212571515112        0.0                   []   8.067212571515112                 nan                nan 17.289245730887043
   6.946275907178013        0.0                   []   6.946275907178013                 nan                nan 18.475896770218082
                 ...        ...                  ...                 ...                 ...                ...                ...
  0.9489406533298774        1.0 [0.8626064417911955]  1.8115470951210728  0.5844278

In [4]:
masses = cluster_table['mass']
level_ages = cluster_table['age']

for i in range(len(level_ages)):
    level_ages[i] = level_ages[i] * 1e6

# convert log ages from log(Myr) to log(yr)
log_ages = np.log10(level_ages)

# print(log_ages)
print("Min log age = " + str(min(log_ages)))
print("Max log age = " + str(max(log_ages)))

# print(level_ages)
print("Min level age = " + str(min(level_ages)))
print("Max level age = " + str(max(level_ages)))

Min log age = 7.138402542266087
Max log age = 7.3011106695566514
Min level age = 13753161.482015437
Max level age = 20003715.51513672


In [5]:
# find index of first binary, create array of companion masses
first_binary = 0
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        first_binary = i
        break
print(first_binary)

compMasses = []
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        compMasses.append(cluster_table['compMass'][i][0])
print(compMasses)

1641
[0.3241624348251086, 0.05095402070602342, 0.06758268036155689, 0.2634619140232143, 3.0305143670780916, 0.019100480058271217, 0.013328533048159051, 0.12267811531436108, 0.16160452370753137, 0.01442445815642728, 0.09685313419544767, 0.02206295254684064, 0.026062091840581544, 0.23206340246858242, 0.010659884204343559, 0.09138690863698644, 0.021026970199695943, 0.07079943158521453, 0.0581860983356197, 0.015522067753595834, 0.11540233656694762, 0.015276826165350291, 0.03731041177184101, 0.25969868292158427, 0.20332028766917049, 0.032323356109109164, 0.18928016428521058, 0.09898339956998825, 0.11899845630244968, 0.21501117944717407, 0.06470821052789688, 0.016764014959335327, 0.04054572198307689, 0.03195300950979017, 0.031136754900217056, 0.09390845096561119, 0.1048092469573021, 0.03370909553992068, 0.13829360753641778, 0.4990076216989321, 0.20981368313804674, 0.12287444910273947, 0.016603971205854686, 0.04813299634079317, 0.23601408835929072, 1.8674397463219425, 0.04728714545608921, 0.0

In [6]:
start = (min(level_ages) // 0.5e6) * 0.5e6
end = (max(level_ages) // 0.5e6) * 0.5e6 + 2 * 0.5e6
increment = 0.5e6  # 500,000

# Create age arrays
level_age_arr = np.arange(start, end, increment)
log_age_arr = np.log10(level_age_arr)

print(log_age_arr)
print(level_age_arr)

[7.13033377 7.14612804 7.161368   7.17609126 7.1903317  7.20411998
 7.21748394 7.23044892 7.24303805 7.25527251 7.26717173 7.2787536
 7.29003461 7.30103    7.31175386]
[13500000. 14000000. 14500000. 15000000. 15500000. 16000000. 16500000.
 17000000. 17500000. 18000000. 18500000. 19000000. 19500000. 20000000.
 20500000.]


In [ ]:
# create isochrone grid - if this is the first time, then this is going to take an hour lmfao
instances = np.empty(len(log_age_arr), dtype=object)

for i in range(len(log_age_arr)):
    my_iso = synthetic.IsochronePhot(log_age_arr[i], AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    instances[i] = my_iso
        
print("isochrone generation done")

Changing to logg=2.50 for T= 13896 logg=2.48
Changing to logg=2.50 for T= 13734 logg=2.46
Changing to logg=2.50 for T= 13574 logg=2.44
Changing to logg=2.50 for T= 13415 logg=2.42
Changing to logg=2.50 for T= 13259 logg=2.41
Changing to logg=2.50 for T= 13104 logg=2.39
Changing to logg=2.50 for T= 12951 logg=2.37
Changing to logg=2.50 for T= 12800 logg=2.35
Changing to logg=2.50 for T= 12651 logg=2.33
Changing to logg=2.50 for T= 12503 logg=2.31
Changing to logg=2.50 for T= 12357 logg=2.29
Changing to logg=2.50 for T= 12213 logg=2.27
Changing to logg=2.50 for T= 12070 logg=2.25
Changing to logg=2.50 for T= 11929 logg=2.23
Changing to logg=2.50 for T= 11789 logg=2.21
Changing to logg=2.00 for T= 10236 logg=1.98
Changing to logg=2.00 for T= 10117 logg=1.96
Changing to logg=2.00 for T=  9998 logg=1.94
Changing to logg=2.00 for T=  9881 logg=1.93
Changing to logg=2.00 for T=  9766 logg=1.91
Changing to logg=2.00 for T=  9651 logg=1.89
Changing to logg=2.00 for T=  9538 logg=1.87
Changing t

In [ ]:
# identify array indeces, create array for labels and colors
idx_arr = range(0, len(log_age_arr))
if len(idx_arr) > 3:
    idx_arr = idx_arr[::2]

# Plot CMD
fig, axes = py.subplots(figsize=(15, 10))
py.subplot(1, 2, 1)
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.legend()

In [ ]:
import sys
sys.path.append('/scratch/wyz5rge/synthetic-hr/12-18')
import interpolatorExp as interpolator

level_ages_myr = level_ages / 1e6
# print(level_ages_myr)

# create array of stars
stars = np.empty(len(cluster_table), dtype=object)

# perform interpolation for each star
for i in range(len(stars)):
    # print(str(i) + " " + str(level_ages_myr[i]) + " " + str(masses[i]))
    stars[i] = interpolator.interpolate(level_ages_myr[i], masses[i], instances, log_age_arr, filters)
    
# convert luminosity values to solar luminosities
watts_to_lsun = 1.0 / (3.846e26) # conversion factor for watts to Lsun

for i in range(len(stars)):
    if stars[i] is None:
        continue
    stars[i][0] = stars[i][0] * watts_to_lsun
    
for i in range(len(stars)):
    print(stars[i])

In [ ]:
fig, axes = py.subplots(figsize=(15, 10))

# plot CMD
py.subplot(1, 2, 1)
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.legend()

for i in range(len(stars)):
    if stars[i] is None:
        continue
    py.plot(stars[i][3] - stars[i][4], stars[i][4], marker='o', markersize=1, color='k')
